
<h1 align="center">SQL Router Query Engine Example</h1>


In [ ]:
!pip install "arize-phoenix[evals,llama-index]" "openai>=1" 'httpx<0.28' gcsfs nest-asyncio "llama-index-readers-wikipedia" "sqlalchemy" wikipedia

In [ ]:
import os
from getpass import getpass

import openai
import pandas as pd
import wikipedia
from llama_index.core import Document, Settings
from llama_index.core.indices import VectorStoreIndex
from llama_index.core.query_engine import NLSQLTableQueryEngine, RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector
from llama_index.core.tools import QueryEngineTool
from llama_index.core.utilities.sql_wrapper import SQLDatabase
from llama_index.llms.openai import OpenAI
from sqlalchemy import (
    create_engine,
    text,
)

import phoenix as px

pd.set_option("display.max_colwidth", 1000)

In [ ]:
(session := px.launch_app()).view()

In [ ]:
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor

from phoenix.otel import register

tracer_provider = register(endpoint="http://127.0.0.1:6006/v1/traces")
LlamaIndexInstrumentor().instrument(skip_dep_check=True, tracer_provider=tracer_provider)

## 3. Configure Your OpenAI API Key

Set your OpenAI API key if it is not already set as an environment variable.

In [ ]:
if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("")
openai.api_key = openai_api_key
os.environ["OPENAI_API_KEY"] = openai_api_key

## 3. Prepare reference data

First, we'll download a dataset that contains technical details of various digital cameras and convert it into an in-memory SQL database. This dataset is provided by Kaggle and more details can be found [here](https://www.kaggle.com/datasets/crawford/1000-cameras-dataset).

In [ ]:
camera_info = pd.read_parquet(
    "https://storage.googleapis.com/arize-phoenix-assets/datasets/structured/camera-info/cameras.parquet"
)

In [ ]:
camera_info.head()

In [ ]:
engine = create_engine("sqlite:///:memory:", future=True)
camera_info.to_sql("cameras", engine, index=False)

In [ ]:
with engine.connect() as connection:
    result = connection.execute(text("SELECT * FROM cameras LIMIT 5")).all()

    for row in result:
        print(row)

Next, for more general queries about digital cameras, we'll download the Wikipedia page on Digital Cameras using the `wikipedia` SDK. We will convert this document into a LlamaIndex `VectorStoreIndex`.

In [ ]:
# load the Digital Camera wikipedia page
page = wikipedia.page(pageid=52797)
doc = Document(id_=page.pageid, text=page.content)

vector_indices = []
vector_index = VectorStoreIndex.from_documents([doc])
vector_indices.append(vector_index)

## 4. Build LlamaIndex Application

Let's use a simple `RouterQueryEngine` using multiple query engine tools. We will either route to the SQL retriever or the vector index built over the "Digital Camera" Wikipedia page.

In [ ]:
Settings.llm = OpenAI(temperature=0.0, model="gpt-4o")

In [ ]:
sql_database = SQLDatabase(engine, include_tables=["cameras"])

sql_query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=["cameras"],
)
sql_tool = QueryEngineTool.from_defaults(
    query_engine=sql_query_engine,
    description=(
        "Useful for translating a natural language query into a SQL query over"
        " a table containing technical details about specific digital camera models: Model,"
        " Release date, Max resolution, Low resolution, Effective pixels, Zoom wide (W),"
        " Zoom tele (T), Normal focus range, Macro focus range, Storage included,"
        " Weight (inc. batteries), Dimensions, Price"
    ),
)

vector_query_engines = [index.as_query_engine() for index in vector_indices]
vector_tools = []
for query_engine in vector_query_engines:
    vector_tool = QueryEngineTool.from_defaults(
        query_engine=query_engine,
        description="Useful for answering generic questions about digital cameras.",
    )
    vector_tools.append(vector_tool)

In [ ]:
query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=([sql_tool] + vector_tools),
)

## 5. Make Queries and Use Phoenix to view Spans

In [ ]:
response = query_engine.query("What is the most expensive digital camera?")
print(str(response))

This query asked for specific details about a camera, and routed to the SQL retriever to get context for the response. The LLM-generated SQL can be seen in a Phoenix span.

![A view of the Phoenix UI showing SQL retrieval](https://storage.googleapis.com/arize-phoenix-assets/assets/docs/notebooks/tracing/llama-index-sql-retrieval-tutorial/sql-retrieval.png)

In [ ]:
response = query_engine.query("Tell me about the history of digital camera sensors.")
print(str(response))

More general queries are routed to the vector index.

![A view of the Phoenix UI showing vector retrieval](https://storage.googleapis.com/arize-phoenix-assets/assets/docs/notebooks/tracing/llama-index-sql-retrieval-tutorial/vectorstoreindex-retrieval.png)

## 6. Final Thoughts

LLM Traces and the accompanying OpenInference Tracing specification is designed to be a category of telemetry data that is used to understand the execution of LLMs and the surrounding application context. This is especially useful when understanding the behavior of more complex RAG applications that might make use of multiple context retrieval strategies, such as mixing a SQL retriever with more-common vector indexes.